# CatBoost on EEG and Spectrogram Data

The engineered features are from this <a href=https://www.kaggle.com/datasets/jacobsharples/engineered-features-for-hms-2024>Kaggle dataset</a> which is built upon three datasets:
* Raw EEG Data
* Raw Spectrogram Data (NAs filled by 0)
* Spectrogram data built by EEGs (<a href=https://www.kaggle.com/datasets/cdeotte/brain-eeg-spectrograms/data)>Spectrogram Data Built by Chris Deotte</a>)

Five classes utilized to build the features:
* **ReadData**: Simple class to read eeg and spectrogram files
* **FeatureEngineerData**: Class that turns data into summarized statistics
* **EEGFeatures**: Feature engineer EEG data
* **SpectrogramFeatures**: Feature engineer Spectrogram data
* **EEGBuiltSpectrogramFeatures**: Feature engineer the EEG-built spectrograms

The EEGFeatures, SpectrogramFeatures, EEGBuiltSpectrogramFeatures classes all contain a method called **get_features()** which contains the engineered features I gather for each dataset.

### Model Used?
Currently using CatBoostClassifier

### Latest Version
* Added a **FEATURES** variable to control which features to train on

In [ ]:
import pandas as pd
import numpy as np
from scipy.special import kl_div
from sklearn.model_selection import GroupKFold
import xgboost as xgb
import catboost as cb
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import sys
sys.path.append('/kaggle/input/kaggle-kl-div/')
from kaggle_kl_div import score

import os
import gc
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

REGENERATING_DATA = False # Whether to regenerate the data, best to keep at false
USE_WEIGHTS = False # Whether or not to use the total votes to weigh the CatBoostClassifier
FEATURES = 'drop' # either all, drop, or select

# Custom Functions and Classes <a class="anchor"  id="classes"></a>

In [ ]:
import pywt, librosa

USE_WAVELET = None 

NAMES = ['LL','LP','RP','RR']

FEATS = [['Fp1','F7','T3','T5','O1'],
         ['Fp1','F3','C3','P3','O1'],
         ['Fp2','F8','T4','T6','O2'],
         ['Fp2','F4','C4','P4','O2']]

# DENOISE FUNCTION
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise(x, wavelet='haar', level=1):    
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    ret=pywt.waverec(coeff, wavelet, mode='per')
    
    return ret

def spectrogram_from_eeg(parquet_path, display=False):
    
    # LOAD MIDDLE 50 SECONDS OF EEG SERIES
    eeg = pd.read_parquet(parquet_path)
    middle = (len(eeg)-10_000)//2
    eeg = eeg.iloc[middle:middle+10_000]
    
    # VARIABLE TO HOLD SPECTROGRAM
    img = np.zeros((128,256,4),dtype='float32')
    
    if display: plt.figure(figsize=(10,7))
    signals = []
    for k in range(4):
        COLS = FEATS[k]
        
        for kk in range(4):
        
            # COMPUTE PAIR DIFFERENCES
            x = eeg[COLS[kk]].values - eeg[COLS[kk+1]].values

            # FILL NANS
            m = np.nanmean(x)
            if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
            else: x[:] = 0

            # DENOISE
            if USE_WAVELET:
                x = denoise(x, wavelet=USE_WAVELET)
            signals.append(x)

            # RAW SPECTROGRAM
            mel_spec = librosa.feature.melspectrogram(y=x, sr=200, hop_length=len(x)//256, 
                  n_fft=1024, n_mels=128, fmin=0, fmax=20, win_length=128)

            # LOG TRANSFORM
            width = (mel_spec.shape[1]//32)*32
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:,:width]

            # STANDARDIZE TO -1 TO 1
            mel_spec_db = (mel_spec_db+40)/40 
            img[:,:,k] += mel_spec_db
                
        # AVERAGE THE 4 MONTAGE DIFFERENCES
        img[:,:,k] /= 4.0
        
        if display:
            plt.subplot(2,2,k+1)
            plt.imshow(img[:,:,k],aspect='auto',origin='lower')
            plt.title(f'EEG {eeg_id} - Spectrogram {NAMES[k]}')
            
    if display: 
        plt.show()
        plt.figure(figsize=(10,5))
        offset = 0
        for k in range(4):
            if k>0: offset -= signals[3-k].min()
            plt.plot(range(10_000),signals[k]+offset,label=NAMES[3-k])
            offset += signals[3-k].max()
        plt.legend()
        plt.title(f'EEG {eeg_id} Signals')
        plt.show()
        print(); print('#'*25); print()
        
    return img

In [ ]:
def kl_div_score(model, X_test, true):
    subm = pd.DataFrame(model.predict_proba(X_test), columns = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote'])
    subm = subm.set_axis(true.columns, axis=1)

    subm['id'] = range(len(subm))
    
    true['id'] = range(len(true))
    return score(true, subm, 'id')

def kl_div_score_reg(model, X_test, true):
    
    preds = model.predict(X_test).clip(0)
    preds = preds/preds.sum(axis=1)[:,None]
    
    subm = pd.DataFrame(preds, columns = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote'])
    subm = subm.set_axis(true.columns, axis=1)

    subm['id'] = range(len(subm))
    
    true['id'] = range(len(true))
    return score(true, subm, 'id')

class ReadData():
    def __init__(self, is_train = True):
        self.is_train = is_train
    
    def _read_data(self, data_type, file_id):
        if self.is_train:
            PATH = f"/kaggle/input/hms-harmful-brain-activity-classification/train_{data_type}/{file_id}.parquet"
        else:
            PATH = f"/kaggle/input/hms-harmful-brain-activity-classification/test_{data_type}/{file_id}.parquet"
        
        return pd.read_parquet(PATH)
        
    def read_spectrogram_data(self, spectrogram_id):
        return self._read_data('spectrograms', spectrogram_id).set_index('time')
    
    def read_eeg_data(self, eeg_id) -> pd.DataFrame:
        return self._read_data('eegs', eeg_id)
    
    def read_eeg_built_spectrogram_data(self, eeg_id) -> pd.DataFrame:
        
        
        montages = ['LL', 'LP', 'RP', 'RR']
        spec = pd.DataFrame()
        
        if self.is_train:
            eeg_specs = np.load(f"/kaggle/input/brain-eeg-spectrograms/EEG_Spectrograms/{eeg_id}.npy")
        else:
            eeg_specs = spectrogram_from_eeg(f"/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/{eeg_id}.parquet")


        for i in range(len(montages)):
            spec = pd.concat([spec, pd.DataFrame(eeg_specs[:,:,i]).T.add_prefix(f'{montages[i]}_')], axis=1)
        
        return spec
    
    def read_train_data(self):
        return pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")
    
    def read_test_data(self):
        return pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/test.csv")

In [ ]:
class FeatureEngineerData(ReadData):
    def __init__(self, metadata, is_train=True, row_id='label_id'):
        '''
        
        Params
        ----------
        metadata : dict
            Contains the information on the eeg ids and labels
        
        '''
        self.metadata = metadata
        self.is_train = is_train
        
        self.row_id = metadata[row_id]    
        
    def get_mean(self, df) -> pd.DataFrame:
        return (df
                .mean()
                .reset_index()
                .set_axis(['var', 'mean'], axis=1)
                .assign(row_id = self.row_id)
                .pivot(columns='var', values='mean', index='row_id')
                .add_prefix('mean_')
        )
     
    def get_max(self, df) -> pd.DataFrame:
        return (df
                .max()
                .reset_index()
                .set_axis(['var', 'max'], axis=1)
                .assign(row_id = self.row_id)
                .pivot(columns='var', values='max', index='row_id')
                .add_prefix('max_')
        )
    
    def get_min(self, df) -> pd.DataFrame:
        return (df
                .max()
                .reset_index()
                .set_axis(['var', 'min'], axis=1)
                .assign(row_id = self.row_id)
                .pivot(columns='var', values='min', index='row_id')
                .add_prefix('min_')
        )   
    
    def get_corr(self, df) -> pd.DataFrame:
        '''
        Returns the correlation of an eeg file
        '''
        def apply_mask(df):
            mask = np.triu(np.ones_like(df, dtype=bool))
            return df.where(mask).unstack().dropna()

        return (df
             .corr()
             .pipe(apply_mask)
             .reset_index()
             .set_axis(['var_1', 'var_2', 'corr'], axis=1)
             .query("var_1 != var_2")
             .assign(
                 row_id = self.row_id,
                 label = lambda x: x.var_1 + "_" + x.var_2
             )
                .pivot(columns='label', values='corr', index='row_id')
                .add_prefix('cor_')
        )
    
    def filter_spectrogram_corr(self, corr_df) -> pd.DataFrame:
        '''
        Returns a dataframe with only the correlation across the same frequency
        '''
        return corr_df[[col for col in corr_df.columns if col.split('_')[2] == col.split('_')[4]]]
    
    def filter_eegspectrogram_corr(self, corr_df) -> pd.DataFrame:
        pass
        
    
    def get_std(self, df) -> pd.DataFrame:
        return (df
                .std()
                .reset_index()
                .set_axis(['var', 'std'], axis=1)
                .assign(row_id = self.row_id)
                .pivot(columns='var', values='std', index='row_id')
                .add_prefix('std_')
        )
    
    def get_range(self, df) -> pd.DataFrame:
        return (
            df
            .max()
            .sub(df.min())
            .reset_index()
            .set_axis(['var', 'range'], axis=1)
            .assign(row_id = self.row_id)
            .pivot(columns='var', values='range', index='row_id')
            .add_prefix('range_')
        )

In [ ]:
class EEGFeatures(FeatureEngineerData):
    
    def get_offset(self):
        if self.metadata.get('right_eeg_index') is None:
            return [0, 10000]
        else:
            return [self.metadata['left_eeg_index'], self.metadata['right_eeg_index']]
        
    def format_eeg_data(self, window_sizes = {}):
        
        offset_range = self.get_offset()
        
        df = self.read_eeg_data(self.metadata['eeg_id']).iloc[offset_range[0]:offset_range[1]]
        
        eeg_df = pd.DataFrame()
        for window in window_sizes:
            left_index = window_sizes[window][0]
            right_index = window_sizes[window][1]
            
            eeg_df = pd.concat([
                eeg_df,
                self.get_features(df.iloc[left_index:right_index], time_id = window)
            ], axis=1)
        
        return eeg_df
    
    def get_features(self, df, time_id) -> pd.DataFrame():
        return (
            pd.concat([
                self.get_mean(df),
                self.get_std(df),
                self.get_max(df),
                self.get_range(df),
                self.get_corr(df)
            ], axis=1).add_prefix(f"eeg_{time_id}_")
        )
    
class SpectrogramFeatures(FeatureEngineerData):
    
    def get_offset(self):
        if self.metadata.get('spectrogram_label_offset_seconds') is None:
            return 0
        else:
            return self.metadata['spectrogram_label_offset_seconds']
    
    def format_spectrogram_data(self, window_sizes = {}):
        
        # Create a variable to make the code more readable
        offset = self.get_offset()
        
        # Read specific spectrogram window
        df = (self.read_spectrogram_data(self.metadata['spectrogram_id'])
              .loc[offset:offset+600]
              .fillna(0)
             )
                
        # Creates the middle of the spectrogram
        middle = (offset+(600+offset))/2

        spec_df = pd.DataFrame()
        for window in window_sizes:
            left_index = window_sizes[window][0]
            right_index = window_sizes[window][1]
                        
            spec_df = pd.concat([
                spec_df,
                self.get_features(df.loc[middle+left_index:middle+right_index], time_id = window)
            ], axis=1)
        
        return spec_df
    
    def get_features(self, df, time_id) -> pd.DataFrame():
        return (
            pd.concat([
                self.get_mean(df),
                self.get_std(df),
                self.get_max(df),
                self.get_min(df),
                self.get_range(df)
            ], axis=1).add_prefix(f"spec_{time_id}_")
        )
    
class EEGBuiltSpectrogramFeatures(FeatureEngineerData):
    def format_custom_spectrogram(self, window_sizes = {()} ):
        
        df = self.read_eeg_built_spectrogram_data(self.metadata['eeg_id']).copy()
        
        spec_df = pd.DataFrame()
        for window in window_sizes:
            left_index = window_sizes[window][0]
            right_index = window_sizes[window][1]
            
            spec_df = pd.concat([
                spec_df,
                self.get_features(df.iloc[left_index:right_index], time_id = window)
            ], axis=1)
            
        return spec_df
    
    def get_features(self, df, time_id) -> pd.DataFrame():
        return (
            pd.concat([
                self.get_mean(df),
                self.get_std(df),
                self.get_max(df),
                self.get_min(df),
                self.get_range(df)
            ], axis=1).add_prefix(f"eegspec_{time_id}_")
        )

# Data

## Window Functions

In [ ]:
eeg_windows = {
    '10s': (4000, 6000), # Middle 10s
    '30s': (2000, 8000), # Middle 30s
    '50s': (0, 10000) # Entire sample (50s)
}

spec_windows = {
    '10m': (-300, 300), # Entire sample
    '5m': (-150, 150),
    '1m': (-30, 30),
    '10s': (-5, 5),
    '20s': (-10, 10),
    '30s': (-15, 15),
    'pre': (-300, -10),
    'post': (10, 300)
    
}

eeg_built_spec_windows = {
    '50s': (0, 256), # Entire sample
    '10s': (100, -100), # 10s
    'pre': (0, 100),
    'post': (-100, 256)
}

In [ ]:
rd = ReadData()

train_df = rd.read_train_data()
train_df['left_eeg_index'] = train_df['eeg_label_offset_seconds'].multiply(200).astype('int')
train_df['right_eeg_index'] = train_df['eeg_label_offset_seconds'].add(50).multiply(200).astype('int')
train_df.head()

## Creating Training Data

In [ ]:
if REGENERATING_DATA:
    df = pd.DataFrame()
    for index, row in tqdm(train_df.query("eeg_sub_id == 0").iterrows()):
        e = EEGFeatures(metadata=dict(row))
        s = SpectrogramFeatures(metadata=dict(row))
        es = EEGBuiltSpectrogramFeatures(metadata=dict(row))

        feature_data = pd.concat([
            e.format_eeg_data(eeg_windows),
            s.format_spectrogram_data(spec_windows),
            es.format_custom_spectrogram(eeg_built_spec_windows)
        ], axis=1)


        df = pd.concat([
            df,
            feature_data
        ])
    print('Finished creating training data...')


In [ ]:
y_prob = train_df.query("eeg_sub_id == 0").filter(like = '_vote')
y_prob = y_prob.divide(y_prob.sum(axis=1), axis=0)
idx = y_prob[y_prob.max(axis=1) == 1].reset_index(drop=True).index

In [ ]:
df = (pd.concat(
        [
        pd.read_parquet("/kaggle/input/engineered-features-for-hms-2024/eeg_data.parquet"),
        pd.read_parquet("/kaggle/input/engineered-features-for-hms-2024/spectrogram_data.parquet"),
        pd.read_parquet("/kaggle/input/engineered-features-for-hms-2024/eeg_built_spectrogram_data.parquet"),
        ], axis=1)
      .sample(frac=1, random_state=42)
     )#.iloc[idx]

X_raw = df.reset_index(drop=True).copy()
y_prob = df.reset_index().rename(columns={'row_id':'label_id'}).merge(train_df).filter(like = '_vote')
y_prob = y_prob.divide(y_prob.sum(axis=1), axis=0)

TARGETS = {
    'Seizure': 0,
    'LPD': 1,
    'GPD': 2,
    'LRDA': 3,
    'GRDA': 4,
    'Other': 5
}
y = df.merge(train_df, left_on='row_id', right_on='label_id')['expert_consensus'].map(TARGETS)

patient_id = train_df.query("eeg_sub_id == 0").iloc[0:len(df)].groupby('eeg_id')['patient_id'].first()

del df
gc.collect()

In [ ]:
# Remove features that are not important in the model

if FEATURES == 'drop':
    drop_cols = []
    drop_cols.extend(X_raw.filter(like='post_range').columns)
    drop_cols.extend(X_raw.filter(like='pre_range').columns)
    drop_cols.extend(X_raw.filter(like='eegspec_post_min').columns)
    drop_cols.extend(X_raw.filter(like='eegspec_post_max').columns)
    drop_cols.extend(X_raw.filter(like='eegspec_post_std').columns)
    drop_cols.extend(X_raw.filter(like='eeg_min').columns)
    drop_cols.extend(X_raw.filter(like='eegspec_pre_').columns)
    drop_cols.extend(X_raw.filter(like='eegspec_post_mean_').columns)
    drop_cols.extend(X_raw.filter(like='eegspec_10s_range').columns)
    drop_cols.extend(X_raw.filter(like='eegspec_50s_range').columns)
    #drop_cols.extend(X_raw.filter(like='eegspec_10s_min').columns)
    #drop_cols.extend(X_raw.filter(like='eegspec_50s_min').columns)
    drop_cols.extend(X_raw.filter(like='eegspec_10s_min_RR').columns)

    X = X_raw[X_raw.columns[~X_raw.columns.isin(drop_cols)]].copy()
    
elif FEATURES == 'select':
    X = X_raw[['eegspec_10s_min_RR_33', 'eegspec_10s_mean_RR_17', 'eegspec_10s_std_LP_65',
        'spec_30s_min_LL_6.64', 'eegspec_10s_std_LL_59', 'spec_1m_max_LP_13.09', 'spec_30s_std_RL_6.45',
      'eeg_50s_range_EKG', 'spec_10s_mean_RL_5.66', 'eegspec_10s_max_LP_31',
      'spec_10s_max_RL_16.8', 'eegspec_50s_std_LP_58', 'spec_20s_max_LP_1.17',
      'eegspec_10s_std_LP_35', 'eegspec_10s_std_RP_86', 'spec_10s_mean_LP_5.66',
      'eegspec_10s_min_LL_29', 'spec_10s_max_RL_2.15', 'spec_10s_max_LP_2.54',
      'spec_1m_max_RL_12.89', 'spec_20s_mean_RL_7.62', 'eegspec_10s_min_RR_50',
      'eegspec_10s_std_LP_34', 'spec_1m_min_RL_11.13', 'eegspec_50s_mean_LP_16',
      'eegspec_pre_range_LP_48', 'eegspec_pre_std_RR_49', 'eeg_10s_cor_T6_T5',
          'eegspec_pre_std_LP_21', 'eeg_30s_cor_C4_C3', 'eegspec_10s_min_RR_41',
          'eegspec_10s_std_LP_32', 'spec_20s_std_LP_1.95', 'eegspec_10s_mean_RR_18',
          'spec_1m_max_LP_18.36', 'spec_5m_std_LP_7.62', 'eegspec_10s_std_LP_92',
          'spec_1m_mean_LP_6.84', 'eegspec_10s_max_RR_35', 'spec_30s_min_RL_7.62',
          'eegspec_50s_std_LL_45', 'eegspec_pre_mean_LL_17', 'eegspec_10s_std_RR_40',
          'eeg_10s_cor_Fz_P3', 'eegspec_10s_std_LP_47', 'eegspec_50s_std_LP_75',
              'spec_20s_mean_RL_8.98', 'spec_20s_min_LP_1.76', 'spec_5m_range_LP_13.28',
              'eegspec_10s_max_RR_17', 'eegspec_10s_max_LL_24', 'eegspec_10s_std_LP_44',
              'spec_1m_std_RP_7.03', 'eegspec_10s_range_RP_52', 'eegspec_10s_max_RR_28', 
               'eegspec_50s_std_LP_39', 'eegspec_10s_std_LP_77']].copy()
else:
    X = X_raw.copy()

del X_raw
gc.collect()

In [ ]:
# Weigh the CatBoostClassifier by the number of votes
total_votes = train_df.filter(like='vote').sum(axis=1).values
max_votes = train_df.filter(like='vote').max(axis=1).values
max_prob = y_prob.max(axis=1)

# Training

In [ ]:
folds = GroupKFold(n_splits=5)
result_dict = {}

for fold, (train_index, valid_index) in enumerate(folds.split(X, y, patient_id)): 
    X_train, y_train = X.iloc[train_index], y.iloc[train_index] 
    X_val, y_val = X.iloc[valid_index], y.iloc[valid_index] 
    
    train_pool = cb.Pool(X_train, y_train,)
    valid_pool = cb.Pool(X_val, y_val)
    
    if USE_WEIGHTS:
        train_pool.set_weight(max_prob[train_index].astype('float'))
        valid_pool.set_weight(max_prob[valid_index].astype('float'))
    
    print('Training: CatBoost...')
    cb_mod = cb.CatBoostClassifier(
        iterations=500,
        max_depth=5,
        objective='MultiClass'
    )
    
    cb_mod.fit(
        train_pool, 
        eval_set=[valid_pool], 
        verbose=50
    )
    
    kl_score = kl_div_score(cb_mod, X_val, y_prob.iloc[valid_index].reset_index(drop=True))
    print('KL-Divergence score:', kl_score)
    
    cb_mod.save_model(f'cb_v{fold}.cat')
    
    result_dict[fold] = {
        'model': cb_mod,
        'kl-score': kl_score,
        'pred': cb_mod.predict_proba(X_val),
        'true_prob':y_prob.iloc[valid_index],
        'true': y_val
    }

In [ ]:
score_list = []
for key in result_dict:
    score_list.append(result_dict[key]['kl-score'])
np.mean(score_list)

# Feature Explanation

In [ ]:
cb_mod.get_feature_importance(prettified=True).head(n=50)

In [ ]:
cb_mod.get_feature_importance(prettified=True).tail(n=50)

In [ ]:
#import shap

In [ ]:
#shap_values = shap.TreeExplainer(cb_mod).shap_values(X_train)
#shap.summary_plot(shap_values, X_train, class_names = y_prob.columns)

# Submission

In [ ]:
rd = ReadData(is_train=False)

test_df = rd.read_test_data()

In [ ]:
X_test = pd.DataFrame()
for index, row in tqdm(test_df.iterrows()):
    e = EEGFeatures(metadata=dict(row), is_train=False, row_id='eeg_id')
    s = SpectrogramFeatures(metadata=dict(row), is_train=False, row_id='eeg_id')
    es = EEGBuiltSpectrogramFeatures(metadata=dict(row), is_train=False, row_id='eeg_id')
    
    feature_data = pd.concat([
        e.format_eeg_data(eeg_windows),
        s.format_spectrogram_data(spec_windows),
        es.format_custom_spectrogram(eeg_built_spec_windows)
    ], axis=1)
    
    
    X_test = pd.concat([
        X_test,
        feature_data
    ])
    
# Select same features from training dataset
X_test = X_test[X_test.columns[X_test.columns.isin(X.columns)]]

In [ ]:
preds = []

for i in range(5):
    print(i,', ',end='')
    model = cb.CatBoostClassifier(
        iterations=1000,
        max_depth=5,
        objective='MultiClass'
    )
    model.load_model(f'cb_v{i}.cat')
    
    test_pool = cb.Pool(
        data = X_test
    )
    
    pred = model.predict_proba(test_pool)
    preds.append(pred)
    
pred = np.mean(preds,axis=0)

In [ ]:
subm = pd.DataFrame(pred, columns=['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote'])
subm['eeg_id'] = test_df.eeg_id

subm[['eeg_id','seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].to_csv("submission.csv", index=False)